In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

In [2]:
hourly_data = pd.read_csv('datasets/hourly_houston_weather.csv').dropna()

In [3]:
hourly_data.head()

,datetime (UTC),"coordinates (lat,lon)",model (name),model elevation (surface),utc_offset (hrs),temperature (degC),dewpoint_temperature (degC),relative_humidity (0-1),wind_speed (m/s),total_cloud_cover (0-1),total_precipitation (mm of water equivalent)
0,2005-01-01 00:00:00,"(29.758938, -95.367697)",era5,23.58,-6.0,20.61,17.70,0.83,2.72,0.53,0.0
1,2005-01-01 01:00:00,"(29.758938, -95.367697)",era5,23.58,-6.0,19.53,16.75,0.84,3.51,0.56,0.0
2,2005-01-01 02:00:00,"(29.758938, -95.367697)",era5,23.58,-6.0,18.76,16.24,0.85,3.80,0.78,0.0
3,2005-01-01 03:00:00,"(29.758938, -95.367697)",era5,23.58,-6.0,18.29,17.02,0.92,3.80,0.85,0.0
4,2005-01-01 04:00:00,"(29.758938, -95.367697)",era5,23.58,-6.0,17.88,16.61,0.92,3.59,0.92,0.0


In [4]:
hourly_data = hourly_data.rename(columns={'datetime (UTC)': 'datetime',
                                          'temperature (degC)': 'temperature',
                                          'dewpoint_temperature (degC)': 'dewpoint_temperature',
                                          'relative_humidity (0-1)': 'relative_humidity',
                                          'wind_speed (m/s)': 'wind_speed',
                                          'total_cloud_cover (0-1)': 'cloud_cover',
                                          'total_precipitation (mm of water equivalent)': 'precipitation'
                                Í          })

In [5]:
# Assuming 'datetime' column is in datetime format
hourly_data['datetime'] = pd.to_datetime(hourly_data['datetime'])

# Extract date from datetime and set it as index
hourly_data['date'] = hourly_data['datetime'].dt.date
hourly_data.set_index('date', inplace=True)

# Define aggregation functions for each column
agg_functions = {'temperature': 'mean',
                 'dewpoint_temperature': 'mean',
                 'relative_humidity': 'mean',
                 'wind_speed': 'mean',
                 'cloud_cover': 'mean',
                 'precipitation': 'sum'}

# Group by date and apply aggregation functions
daily_data = hourly_data.groupby('date').agg(agg_functions)

# Reset the index to get 'date' back as a column
daily_data.reset_index(inplace=True)


daily_data

,date,temperature,dewpoint_temperature,relative_humidity,wind_speed,cloud_cover,precipitation
0,2005-01-01,19.346250,16.935000,0.865417,3.145833,0.801250,0.25
1,2005-01-02,20.988750,19.326250,0.906250,4.716667,0.899583,0.51
2,2005-01-03,21.963750,19.431250,0.862500,5.099167,0.904167,0.29
3,2005-01-04,21.030000,18.850000,0.878333,4.538333,0.887917,0.51
4,2005-01-05,21.838750,18.479167,0.822083,5.390417,0.813333,0.47
...,...,...,...,...,...,...,...
6569,2022-12-27,6.512500,-1.231667,0.596667,2.306250,0.014583,0.00
6570,2022-12-28,11.225417,6.572500,0.751250,3.484167,0.433333,0.06
6571,2022-12-29,21.205833,18.112500,0.836250,5.069583,0.508750,7.53
6572,2022-12-30,18.987917,16.593750,0.865833,2.800000,0.804167,12.48


In [6]:
daily_data.precipitation.describe()

count    6574.000000
mean        3.407919
std         8.606541
min         0.000000
25%         0.000000
50%         0.380000
75%         2.600000
max       155.960000
Name: precipitation, dtype: float64

In [7]:
# round down to 0 if precipitation is less than 1 mm
daily_data['precipitation'] = daily_data['precipitation'].apply(lambda x: max(0, 0 if x < 3 else x))


In [8]:
daily_data.precipitation.value_counts()

0.00     5049
6.30        5
3.47        4
7.57        4
4.34        4
         ... 
32.54       1
10.13       1
6.75        1
23.59       1
16.74       1
Name: precipitation, Length: 1305, dtype: int64

In [9]:
daily_data['rain_tomorrow'] = (daily_data['precipitation'].shift(-1) > 0).astype(int)

In [10]:
daily_data.to_csv('datasets/daily_houston_data.csv', index=False)